In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pathlib import Path

from PIL import Image

In [ ]:
!pip install utils

In [ ]:
import utils

In [ ]:
!nvidia-smi

Wed Nov  1 06:18:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
from enum import Enum

import ipywidgets as widgets
from IPython.display import display


class Algorithms(Enum):
    ALL = "all"
    WAIFU2X = "waifu2x"
    SRMD = "srmd"
    REALSR = "realsr"
    RIFE = "rife"
    REALCUGAN = "realcugan"
    ANIME4K = "anime4k"


selected_algorithms = ["smrd"]


def confirm_choice(b):
    global selected_algorithms
    selected_algorithms = []

    if checkboxes[0].value:
        selected_algorithms.append(Algorithms.ALL)
        if any(
            [box.value for box in checkboxes[1:]]
        ):
            print("Warning: 'all' selected, overriding other selections.")
    else:
        for box, algorithm in zip(checkboxes[1:], list(Algorithms)[1:]):
            if box.value:
                selected_algorithms.append(algorithm)

    print(
        f"Selection confirmed: {', '.join([algo.value for algo in selected_algorithms])}"
    )


checkboxes = [
    widgets.Checkbox(value=False, description=algorithm.value)
    for algorithm in Algorithms
]
confirm_button = widgets.Button(description="Confirm")
confirm_button.on_click(confirm_choice)
ui = widgets.VBox(checkboxes + [confirm_button])
display(ui)

Selection confirmed: srmd


In [17]:
#@title ## Step 2: Install Video2X and Dependencies

#@markdown **This step installs Video2X and its dependencies.**

#@markdown Since some dependencies must be compiled from source code, it may take a while to complete.

# get the optional dependencies to install
if 'selected_algorithms' not in globals() or selected_algorithms is None:
    raise NameError("No algorithms selected. Did you forget to run the previous step?")
algorithms = ','.join([a.value for a in selected_algorithms])

!apt-get update
!apt-get install -y --no-install-recommends \
    python3-pip libvulkan-dev glslang-dev glslang-tools \
    build-essential swig ninja-build ffmpeg nvidia-driver-535 \
    mpv xserver-xorg-video-dummy xvfb

# temporary: install video2x directly from GitHub
#   since version 5.0.0 doesn't have a release on PyPI yet
!pip install -U pip wheel pdm-pep517 setuptools setuptools-scm
!pip install -Uv 'video2x[{algorithms}]@git+https://github.com/k4yt3x/video2x.git@b382f39'

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 3s (77.5 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libvulkan-dev is already t

In [5]:
#@title ## Step 3: Restart Runtime

#@markdown **The runtime must be restarted after installing Video2X** to reload the kernel drivers.

#@markdown Execute this cell to restart the runtime. It's normal that you get an error saying the session crashed.

exit()

In [6]:
#@title ## Step 4: Select/Upload Input File

#@markdown **Select or upload the file to process.**

#@markdown Google Drive will be used for storage so it will be mounted in this step.

import pathlib

import ipywidgets as widgets
from google.colab import drive, files
from IPython.display import display

drive.mount("/content/drive")
gdrive_path = pathlib.Path("/content/drive/MyDrive")

input_file = "/content/drive/MyDrive/Colab_Notebooks/test1.mp4"

def on_dropdown_change(change):
    global input_file
    if change["name"] == "value":
        input_file = pathlib.Path(change["new"])
        print(f"Selected input file: {input_file}")

def upload_new_file(button):
    global input_file
    uploaded = files.upload()

    # Check if the user canceled the upload
    if not uploaded:
        print("Upload canceled.")
        return

    for file_name in uploaded.keys():
        input_file = pathlib.Path("/content") / file_name
        print(f"Uploaded and set input file: {input_file}")

file_dropdown = widgets.Dropdown(
    options=list(gdrive_path.iterdir()), description="Select file:"
)
file_dropdown.observe(on_dropdown_change)
display(file_dropdown)

upload_button = widgets.Button(description="Upload New File")
upload_button.on_click(upload_new_file)
display(upload_button)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Dropdown(description='Select file:', options=(PosixPath('/content/drive/MyDrive/How to get started with Drive.…

Button(description='Upload New File', style=ButtonStyle())

Selected input file: /content/drive/MyDrive/test1.mp4


In [7]:
#@title ## Step 5.1: Upscale


import contextlib
import os
import pathlib
import subprocess
import sys
import time

import ipywidgets as widgets
import psutil
from google.colab import drive, files
from IPython.display import clear_output, display
from video2x import Video2X

# launch dummy Xorg server for MPV GPU
# write Xorg config
dummy_xorg_conf = """
Section "Device"
    Identifier  "Configured Video Device"
    Driver      "dummy"
EndSection

Section "Monitor"
    Identifier  "Configured Monitor"
EndSection

Section "Screen"
    Identifier  "Default Screen"
    Monitor     "Configured Monitor"
    Device      "Configured Video Device"
EndSection
"""
with open('/etc/X11/xorg.conf', 'w') as f:
    f.write(dummy_xorg_conf)

# start Xorg
for proc in psutil.process_iter():
    with contextlib.suppress(psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
        if proc.name() == 'Xorg':
            print("Killing existing Xorg process")
            proc.kill()
            time.sleep(0.5)
print("Starting dummy Xorg")
subprocess.Popen(["Xorg", ":0"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# export environment variable for subprocesses
%env DISPLAY=:0

if 'input_file' not in globals() or input_file is None:
    raise NameError("Input file not specified. Did you forget to run the previous step?")

if 'gdrive_path' not in globals():
    raise NameError("Google Drive path not specified. Did you forget to run the previous step?")



# The algorithm to use. Refer to the [Wiki page](https://github.com/k4yt3x/video2x/wiki/Algorithms) for more information.
algorithm = "srmd" #@param ["anime4k", "waifu2x", "srmd", "realsr", "realcugan"]

# Width and height of the output video.
width = 406 #@param {type:"number"}
height = 720 #@param {type:"number"}


# The de-noise level. Different models support different values. If setting the value result in an error, try lowering the value.
noise = 3 #@param {type:"slider", min:0, max:3, step:1}

# Number of parallel processes to launch. There usually isn't a point setting this higher than 3 as the performance wouldn't improve.
processes = 1 #@param {type:"slider", min:1, max:10, step:1}

# Adjacent frames with < n% diff won't be processed.
threshold = 0 #@param {type:"slider", min:0, max:100, step:1}

if width == 0 and height == 0:
    raise ValueError("You must specify either output width or height")

output_file = gdrive_path / f"{input_file.stem}_{algorithm}_{width}x{height}{input_file.suffix}"
if output_file.exists():
    raise FileExistsError(f"Cannot continue: file {output_file} already exists")
print(f"Output will be written to: {output_file}")

video2x = Video2X()
video2x.upscale(
    input_file,
    output_file,
    width,
    height,
    noise,
    processes,
    threshold,
    algorithm,
)

print("Video processing completed successfully")

Killing existing Xorg process


2023-11-01 09:10:22.087 | INFO     | video2x.video2x:_get_video_info:123 - Reading input video information


Starting dummy Xorg
env: DISPLAY=:0
Output will be written to: /content/drive/MyDrive/test1_srmd_406x720.mp4


09:10:23.228509 | INFO     | Starting video decoder

09:10:23.239610 | INFO     | Starting video encoder

...

head = self.prepare_header(etype, self.long_header)

RecursionError: maximum recursion depth exceeded

09:13:04.339774 | INFO     | Processing has completed

09:13:04.344547 | INFO     | Writing video trailer

Video processing completed successfully


In [ ]:
#@title ## Step 5.2: Interpolate



import pathlib
from video2x import Video2X

video2x = Video2X()

# The input and output files' file names. Video2X will operate in the root directory of your Google Drive.
input_filename = "input.mp4" #@param {type:"string"}
output_filename = "output.mp4" #@param {type:"string"}

# Number of parallel processes to launch. There usually isn't a point setting this higher than 3 as the performance wouldn't improve.
processes = 1 #@param {type:"slider", min:1, max:10, step:1}

# Adjacent frames with > n% diff won't be processed.
threshold = 5 #@param {type:"slider", min:0, max:100, step:1}

# The algorithm to use. Refer to the [Wiki page](https://github.com/k4yt3x/video2x/wiki/Algorithms) for more information.
algorithm = "rife" #@param ["rife"]

# check if input file exists
gdrive = pathlib.Path("/mnt/gdrive/MyDrive/")
if not (gdrive / input_filename).is_file():
    raise FileNotFoundError(f"file {(gdrive / input_filename)} does not exist")

video2x.interpolate(
    gdrive / input_filename,
    gdrive / output_filename,
    processes,
    threshold,
    algorithm,
)